# Using a DB in sqlite3


## SQL in sqlite3

In [2]:
%%capture
%%bash
apt-get update
apt-get install -y sqlite3


In [3]:
!sqlite3 --help


Usage: sqlite3 [OPTIONS] FILENAME [SQL]
FILENAME is the name of an SQLite database. A new database is created
if the file does not previously exist.
OPTIONS include:
   -A ARGS...           run ".archive ARGS" and exit
   -append              append the database to the end of the file
   -ascii               set output mode to 'ascii'
   -bail                stop after hitting an error
   -batch               force batch I/O
   -box                 set output mode to 'box'
   -column              set output mode to 'column'
   -cmd COMMAND         run "COMMAND" before reading stdin
   -csv                 set output mode to 'csv'
   -deserialize         open the database using sqlite3_deserialize()
   -echo                print commands before execution
   -init FILENAME       read/process named file
   -[no]header          turn headers on or off
   -help                show this message
   -html                set output mode to HTML
   -interactive         force interactive I/O
   -j

In [4]:
!sqlite3 --version

3.37.2 2022-01-06 13:25:41 872ba256cbf61d9290b571c0e6d82a20c224ca3ad82971edc46b29818d5dalt1


## Sample database

The [Chinook sample](https://www.sqlitetutorial.net/sqlite-sample-database/)

- Zip file of [sqlite DB](https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip)

- [ER diagram](https://www.sqlitetutorial.net/wp-content/uploads/2018/03/sqlite-sample-database-diagram-color.pdf)

- [ER symbols/meanings](https://d2slcw3kip6qmk.cloudfront.net/marketing/pages/chart/erd-symbols/ERD-Notation.PNG)

- [Draw IO](https://app.diagrams.net/) for creating ER diagrams, flowcharts, etc.

In [5]:
%%bash
[ -f chinook.zip ] ||
  curl -s -O https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip
unzip -l chinook.zip


Archive:  chinook.zip
  Length      Date    Time    Name
---------  ---------- -----   ----
   884736  2015-11-29 10:53   chinook.db
---------                     -------
   884736                     1 file


In [6]:
!unzip -u chinook.zip


Archive:  chinook.zip
  inflating: chinook.db              


In [7]:
!ls -la

total 1180
drwxr-xr-x 1 root root   4096 Oct 17 19:23 .
drwxr-xr-x 1 root root   4096 Oct 17 19:18 ..
-rw-r--r-- 1 root root 884736 Nov 29  2015 chinook.db
-rw-r--r-- 1 root root 305596 Oct 17 19:23 chinook.zip
drwxr-xr-x 4 root root   4096 Oct 14 13:22 .config
drwxr-xr-x 1 root root   4096 Oct 14 13:23 sample_data


In [8]:
# Get a list of the tables in the database
%%script sqlite3 --column --header chinook.db
.tables


albums          employees       invoices        playlists     
artists         genres          media_types     tracks        
customers       invoice_items   playlist_track


In [9]:
# Show the schema for the entire database
%%script sqlite3 --column --header chinook.db
.schema


CREATE TABLE IF NOT EXISTS "albums"
(
    [AlbumId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    [Title] NVARCHAR(160)  NOT NULL,
    [ArtistId] INTEGER  NOT NULL,
    FOREIGN KEY ([ArtistId]) REFERENCES "artists" ([ArtistId]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION
);
CREATE TABLE sqlite_sequence(name,seq);
CREATE TABLE IF NOT EXISTS "artists"
(
    [ArtistId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    [Name] NVARCHAR(120)
);
CREATE TABLE IF NOT EXISTS "customers"
(
    [CustomerId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    [FirstName] NVARCHAR(40)  NOT NULL,
    [LastName] NVARCHAR(20)  NOT NULL,
    [Company] NVARCHAR(80),
    [Address] NVARCHAR(70),
    [City] NVARCHAR(40),
    [State] NVARCHAR(40),
    [Country] NVARCHAR(40),
    [PostalCode] NVARCHAR(10),
    [Phone] NVARCHAR(24),
    [Fax] NVARCHAR(24),
    [Email] NVARCHAR(60)  NOT NULL,
    [SupportRepId] INTEGER,
    FOREIGN KEY ([SupportRepId]) REFERENCES "employees" ([EmployeeId]) 
		ON DELETE NO ACTION ON 

In [10]:
# Select the first 10 entries from the employees table
%%script sqlite3 --column --header chinook.db
select *
from employees
limit 10


EmployeeId  LastName  FirstName  Title                ReportsTo  BirthDate            HireDate             Address                      City        State  Country  PostalCode  Phone              Fax                Email                   
----------  --------  ---------  -------------------  ---------  -------------------  -------------------  ---------------------------  ----------  -----  -------  ----------  -----------------  -----------------  ------------------------
1           Adams     Andrew     General Manager                 1962-02-18 00:00:00  2002-08-14 00:00:00  11120 Jasper Ave NW          Edmonton    AB     Canada   T5K 2N1     +1 (780) 428-9482  +1 (780) 428-3457  andrew@chinookcorp.com  
2           Edwards   Nancy      Sales Manager        1          1958-12-08 00:00:00  2002-05-01 00:00:00  825 8 Ave SW                 Calgary     AB     Canada   T2P 2T3     +1 (403) 262-3443  +1 (403) 262-3322  nancy@chinookcorp.com   
3           Peacock   Jane       Sales Suppo

In [11]:
# Count the number of unique albums
%%script sqlite3 --column --header chinook.db
select count(distinct AlbumID)
from tracks


count(distinct AlbumID)
-----------------------
347                    


In [12]:
# Select the first 10 entries from the albums table
%%script sqlite3 --column --header chinook.db
select *
from albums
limit 10


AlbumId  Title                                  ArtistId
-------  -------------------------------------  --------
1        For Those About To Rock We Salute You  1       
2        Balls to the Wall                      2       
3        Restless and Wild                      2       
4        Let There Be Rock                      1       
5        Big Ones                               3       
6        Jagged Little Pill                     4       
7        Facelift                               5       
8        Warner 25 Anos                         6       
9        Plays Metallica By Four Cellos         7       
10       Audioslave                             8       


In [13]:
# Select the first 10 entries from the artists table
%%script sqlite3 --column --header chinook.db
select *
from artists
limit 10


ArtistId  Name                
--------  --------------------
1         AC/DC               
2         Accept              
3         Aerosmith           
4         Alanis Morissette   
5         Alice In Chains     
6         Antônio Carlos Jobim
7         Apocalyptica        
8         Audioslave          
9         BackBeat            
10        Billy Cobham        


In [14]:
# Select the first 10 entries from joining the artists and albums tables
%%script sqlite3 --column --header chinook.db
select *
from artists
join albums
on artists.ArtistID = albums.ArtistID
limit 10


ArtistId  Name                  AlbumId  Title                                  ArtistId
--------  --------------------  -------  -------------------------------------  --------
1         AC/DC                 1        For Those About To Rock We Salute You  1       
2         Accept                2        Balls to the Wall                      2       
2         Accept                3        Restless and Wild                      2       
1         AC/DC                 4        Let There Be Rock                      1       
3         Aerosmith             5        Big Ones                               3       
4         Alanis Morissette     6        Jagged Little Pill                     4       
5         Alice In Chains       7        Facelift                               5       
6         Antônio Carlos Jobim  8        Warner 25 Anos                         6       
7         Apocalyptica          9        Plays Metallica By Four Cellos         7       
8         Audioslave 

In [15]:
# Select albums by AC/DC
%%script sqlite3 --column --header chinook.db
select *
from artists
join albums
on artists.ArtistID = albums.ArtistID
where Name = 'AC/DC'
limit 10


ArtistId  Name   AlbumId  Title                                  ArtistId
--------  -----  -------  -------------------------------------  --------
1         AC/DC  1        For Those About To Rock We Salute You  1       
1         AC/DC  4        Let There Be Rock                      1       


In [16]:
# Select some fields from the employees table
%%script sqlite3 --column --header chinook.db
select EmployeeId, LastName, FirstName, Title, ReportsTo
from employees


EmployeeId  LastName  FirstName  Title                ReportsTo
----------  --------  ---------  -------------------  ---------
1           Adams     Andrew     General Manager               
2           Edwards   Nancy      Sales Manager        1        
3           Peacock   Jane       Sales Support Agent  2        
4           Park      Margaret   Sales Support Agent  2        
5           Johnson   Steve      Sales Support Agent  2        
6           Mitchell  Michael    IT Manager           1        
7           King      Robert     IT Staff             6        
8           Callahan  Laura      IT Staff             6        


In [17]:
# Select employees and their bosses
%%script sqlite3 --column --header chinook.db
select emp1.EmployeeId, emp1.LastName, emp1.FirstName, emp1.Title, emp1.ReportsTo, emp2.ReportsTo, emp2.EmployeeId, emp2.LastName, emp2.FirstName
from employees as emp1
join employees as emp2
on emp1.ReportsTo = emp2.EmployeeID


EmployeeId  LastName  FirstName  Title                ReportsTo  ReportsTo  EmployeeId  LastName  FirstName
----------  --------  ---------  -------------------  ---------  ---------  ----------  --------  ---------
2           Edwards   Nancy      Sales Manager        1                     1           Adams     Andrew   
3           Peacock   Jane       Sales Support Agent  2          1          2           Edwards   Nancy    
4           Park      Margaret   Sales Support Agent  2          1          2           Edwards   Nancy    
5           Johnson   Steve      Sales Support Agent  2          1          2           Edwards   Nancy    
6           Mitchell  Michael    IT Manager           1                     1           Adams     Andrew   
7           King      Robert     IT Staff             6          1          6           Mitchell  Michael  
8           Callahan  Laura      IT Staff             6          1          6           Mitchell  Michael  


In [18]:
# The first three entries from a self cross join of employees
%%script sqlite3 --column --header chinook.db
select *
from employees as emp1
join employees as emp2
limit 3

EmployeeId  LastName  FirstName  Title            ReportsTo  BirthDate            HireDate             Address              City      State  Country  PostalCode  Phone              Fax                Email                   EmployeeId  LastName  FirstName  Title                ReportsTo  BirthDate            HireDate             Address              City      State  Country  PostalCode  Phone              Fax                Email                 
----------  --------  ---------  ---------------  ---------  -------------------  -------------------  -------------------  --------  -----  -------  ----------  -----------------  -----------------  ----------------------  ----------  --------  ---------  -------------------  ---------  -------------------  -------------------  -------------------  --------  -----  -------  ----------  -----------------  -----------------  ----------------------
1           Adams     Andrew     General Manager             1962-02-18 00:00:00  2002-08-14 00:00

In [19]:
# Select a few columns from a self cross joins of employees
%%script sqlite3 --column --header chinook.db
select emp1.EmployeeID, emp1.FirstName, emp1.ReportsTo, emp2.EmployeeID, emp2.FirstName
from employees as emp1
join employees as emp2


EmployeeId  FirstName  ReportsTo  EmployeeId  FirstName
----------  ---------  ---------  ----------  ---------
1           Andrew                1           Andrew   
1           Andrew                2           Nancy    
1           Andrew                3           Jane     
1           Andrew                4           Margaret 
1           Andrew                5           Steve    
1           Andrew                6           Michael  
1           Andrew                7           Robert   
1           Andrew                8           Laura    
2           Nancy      1          1           Andrew   
2           Nancy      1          2           Nancy    
2           Nancy      1          3           Jane     
2           Nancy      1          4           Margaret 
2           Nancy      1          5           Steve    
2           Nancy      1          6           Michael  
2           Nancy      1          7           Robert   
2           Nancy      1          8           La

In [20]:
# Select the first 10 entries from a self cross joins of employees
%%script sqlite3 --column --header chinook.db
select emp1.EmployeeID, emp1.FirstName, emp1.ReportsTo, emp2.EmployeeID, emp2.FirstName
from employees as emp1
join employees as emp2
limit 10

EmployeeId  FirstName  ReportsTo  EmployeeId  FirstName
----------  ---------  ---------  ----------  ---------
1           Andrew                1           Andrew   
1           Andrew                2           Nancy    
1           Andrew                3           Jane     
1           Andrew                4           Margaret 
1           Andrew                5           Steve    
1           Andrew                6           Michael  
1           Andrew                7           Robert   
1           Andrew                8           Laura    
2           Nancy      1          1           Andrew   
2           Nancy      1          2           Nancy    


In [21]:
# Select employees and their supervisors
%%script sqlite3 --column --header chinook.db
select emp1.EmployeeID, emp1.FirstName, emp1.ReportsTo, emp2.EmployeeID, emp2.FirstName
from employees as emp1
join employees as emp2
where emp1.ReportsTo = emp2.EmployeeID
order by emp1.EmployeeID

EmployeeId  FirstName  ReportsTo  EmployeeId  FirstName
----------  ---------  ---------  ----------  ---------
2           Nancy      1          1           Andrew   
3           Jane       2          2           Nancy    
4           Margaret   2          2           Nancy    
5           Steve      2          2           Nancy    
6           Michael    1          1           Andrew   
7           Robert     6          6           Michael  
8           Laura      6          6           Michael  


### Building a complex join using a CTE


In [22]:
# Select customers, their rep, and the rep's supervisor
## uses CTEs
%%script sqlite3 --column --header chinook.db

with
emp as (
  select EmployeeID, FirstName, Lastname, ReportsTo
  from employees
)

select *
from emp
limit 10



EmployeeID  FirstName  Lastname  ReportsTo
----------  ---------  --------  ---------
1           Andrew     Adams              
2           Nancy      Edwards   1        
3           Jane       Peacock   2        
4           Margaret   Park      2        
5           Steve      Johnson   2        
6           Michael    Mitchell  1        
7           Robert     King      6        
8           Laura      Callahan  6        


In [23]:
# Select customers, their rep, and the rep's supervisor
## uses CTEs
%%script sqlite3 --column --header chinook.db

with
emp as (
  select EmployeeID, FirstName, Lastname, ReportsTo
  from employees
),
reps as (
  select *
  from emp
)

select *
from reps


EmployeeID  FirstName  Lastname  ReportsTo
----------  ---------  --------  ---------
1           Andrew     Adams              
2           Nancy      Edwards   1        
3           Jane       Peacock   2        
4           Margaret   Park      2        
5           Steve      Johnson   2        
6           Michael    Mitchell  1        
7           Robert     King      6        
8           Laura      Callahan  6        


In [24]:
# Select customers, their rep, and the rep's supervisor
## uses CTEs
%%script sqlite3 --column --header chinook.db

with
emp as (
  select EmployeeID, FirstName, Lastname, ReportsTo
  from employees
),
reps as (
  select EmployeeID, FirstName, Lastname, ReportsTo
  from emp
),
supervisors as (
  select EmployeeID, FirstName, Lastname
  from emp
)

select *
from supervisors



EmployeeID  FirstName  Lastname
----------  ---------  --------
1           Andrew     Adams   
2           Nancy      Edwards 
3           Jane       Peacock 
4           Margaret   Park    
5           Steve      Johnson 
6           Michael    Mitchell
7           Robert     King    
8           Laura      Callahan


In [25]:
# Select customers, their rep, and the rep's supervisor
## uses CTEs
%%script sqlite3 --column --header chinook.db

with
emp as (
  select EmployeeID, FirstName, Lastname, ReportsTo
  from employees
),
reps as (
  select EmployeeID, FirstName, Lastname, ReportsTo
  from emp
),
supervisors as (
  select EmployeeID, FirstName, Lastname
  from emp
),
workers as (
  select r.EmployeeID as "EmpID", r.Firstname as "Rep", r.ReportsTo as "ReportID", s.EmployeeID as "SuperID", s.FirstName as "Supervisor"
  from reps as r
  join supervisors as s
  where r.ReportsTo = s.EmployeeID
)

select *
from workers




EmpID  Rep       ReportID  SuperID  Supervisor
-----  --------  --------  -------  ----------
2      Nancy     1         1        Andrew    
3      Jane      2         2        Nancy     
4      Margaret  2         2        Nancy     
5      Steve     2         2        Nancy     
6      Michael   1         1        Andrew    
7      Robert    6         6        Michael   
8      Laura     6         6        Michael   


In [26]:
# Select customers, their rep, and the rep's supervisor
## uses CTEs
%%script sqlite3 --column --header chinook.db

with
emp as (
  select EmployeeID, FirstName, Lastname, ReportsTo
  from employees
),
reps as (
  select EmployeeID, FirstName, Lastname, ReportsTo
  from emp
),
supervisors as (
  select EmployeeID, FirstName, Lastname
  from emp
),
workers as (
  select r.EmployeeID as "EmpID", r.Firstname as "Rep", r.ReportsTo as "ReportID", s.EmployeeID as "SuperID", s.FirstName as "Supervisor"
  from reps as r
  join supervisors as s
  where r.ReportsTo = s.EmployeeID
),
cust as (
  select CustomerID, FirstName as "Customer", SupportRepID
  from customers
)

select *
from cust
limit 10

CustomerID  Customer   SupportRepID
----------  ---------  ------------
1           Luís       3           
2           Leonie     5           
3           François   3           
4           Bjørn      4           
5           František  4           
6           Helena     5           
7           Astrid     5           
8           Daan       4           
9           Kara       4           
10          Eduardo    4           


In [27]:
# Select customers, their rep, and the rep's supervisor
## uses CTEs
%%script sqlite3 --column --header chinook.db

with
emp as (
  select EmployeeID, FirstName, Lastname, ReportsTo, Title
  from employees
),
reps as (
  select EmployeeID, FirstName, Lastname, ReportsTo
  from emp
  where Title = "Sales Support Agent"
),
supervisors as (
  select EmployeeID, FirstName, Lastname
  from emp
  where Title != "Sales Support Agent"

),
workers as (
  select r.EmployeeID as "EmpID", r.Firstname as "Rep", r.ReportsTo as "ReportID", s.EmployeeID as "SuperID", s.FirstName as "Supervisor"
  from reps as r
  join supervisors as s
  where r.ReportsTo = s.EmployeeID
),
cust as (
  select CustomerID, FirstName as "Customer", SupportRepID
  from customers
),
cust_rep_super as (
  select Customer, Rep, Supervisor
  from cust
  join workers
  where SupportRepID = EmpID
)

SELECT *
FROM cust_rep_super
limit 10


Customer  Rep   Supervisor
--------  ----  ----------
Luís      Jane  Nancy     
François  Jane  Nancy     
Roberto   Jane  Nancy     
Jennifer  Jane  Nancy     
Michelle  Jane  Nancy     
Tim       Jane  Nancy     
Frank     Jane  Nancy     
Robert    Jane  Nancy     
Edward    Jane  Nancy     
Ellie     Jane  Nancy     


## You Turn

- How many unique albums are in the Tracks table?
- How many tracks are on the album with AlbumID 1?
- Do the totals in the Invoices table add up to the lineitems in the Invoice_Items table?


In [28]:
# How many unique records are in the Tracks table?
%%script sqlite3 --column --header chinook.db
select count(distinct AlbumID) as unique_albums
from `tracks`
;


unique_albums
-------------
347          


In [29]:
# How many unique records are in the Tracks table?
%%script sqlite3 --column --header chinook.db
select count(AlbumID)
from tracks
;

count(AlbumID)
--------------
3503          


In [30]:
# How many tracks are on the album with AlbumID 1?
%%script sqlite3 --column --header chinook.db
select count(1) as track_count
from tracks
where albumid = 1
;

track_count
-----------
10         


In [31]:
# Do the totals in the Invoices table add up to the lineitems in the Invoice_Items table?
%%script sqlite3 --column --header chinook.db
select InvoiceID, Total
from invoices
limit 10


InvoiceId  Total
---------  -----
1          1.98 
2          3.96 
3          5.94 
4          8.91 
5          13.86
6          0.99 
7          1.98 
8          1.98 
9          3.96 
10         5.94 


In [32]:
# Do the totals in the Invoices table add up to the lineitems in the Invoice_Items table?
%%script sqlite3 --column --header chinook.db
select invoiceID, sum((unitprice * quantity)) as line_total
from invoice_items
group by invoiceid
limit 10


InvoiceId  line_total
---------  ----------
1          1.98      
2          3.96      
3          5.94      
4          8.91      
5          13.86     
6          0.99      
7          1.98      
8          1.98      
9          3.96      
10         5.94      


In [33]:
%%script sqlite3 --column --header chinook.db
with
invoices_ as (
  select InvoiceID, Total as Total
  from invoices
),
line_items as (
  select invoiceID, sum((unitprice * quantity)) as line_total
  from invoice_items
  group by invoiceid
),
side_by_side as (
  select *
  from invoices_ as i
  join line_items as li
  on i.invoiceid = li.invoiceid
)
select * from side_by_side
where total != line_total
limit 10




InvoiceID  Total  invoiceID:1  line_total
---------  -----  -----------  ----------
5          13.86  5            13.86     
12         13.86  12           13.86     
19         13.86  19           13.86     
26         13.86  26           13.86     
33         13.86  33           13.86     
40         13.86  40           13.86     
47         13.86  47           13.86     
54         13.86  54           13.86     
61         13.86  61           13.86     
68         13.86  68           13.86     


In [38]:
%%script sqlite3 --column --header chinook.db
with track_info as (
    select
        name as trackname,
        composer,
        genreid
    from
        tracks
)
select
    ti.composer,
    g.name as genre,
    count(*) as trackcount
from
    track_info ti
join
    genres g on ti.genreid = g.genreid
where
    ti.composer is not null
group by
    ti.composer, g.name
order by
    trackcount desc
limit 50;

composer                                                             genre               trackcount
-------------------------------------------------------------------  ------------------  ----------
U2                                                                   Rock                44        
Steve Harris                                                         Metal               36        
Jagger/Richards                                                      Rock                35        
Billy Corgan                                                         Alternative & Punk  31        
Kurt Cobain                                                          Rock                26        
Steve Harris                                                         Rock                26        
The Tea Party                                                        Alternative & Punk  24        
Chico Science                                                        Latin               23        


In [34]:
%%script sqlite3 --column --header chinook.db
SELECT
    CASE
        WHEN al.ArtistID IS NULL THEN 'No Artist'
        ELSE 'With Artist'
    END AS AlbumStatus,
    COUNT(*) AS AlbumCount
FROM
    albums al
LEFT JOIN
    artists ar ON al.ArtistID = ar.ArtistID
GROUP BY
    CASE
        WHEN al.ArtistID IS NULL THEN 'No Artist'
        ELSE 'With Artist'
    END;

AlbumStatus  AlbumCount
-----------  ----------
With Artist  347       


In [35]:
%%script sqlite3 --column --header chinook.db
SELECT
    CASE
        WHEN al.ArtistID IS NULL THEN 'Without Artist'
        ELSE 'With Artist'
    END AS AlbumStatus,
    COUNT(*) AS AlbumCount
FROM
    albums al
LEFT JOIN
    artists ar ON al.ArtistID = ar.ArtistID
GROUP BY
    CASE
        WHEN al.ArtistID IS NULL THEN 'Without Artist'
        ELSE 'With Artist'
    END

UNION ALL

SELECT NULL, NULL

UNION ALL

SELECT 'Albums Without Artists', COUNT(*) AS AlbumCount
FROM albums
WHERE ArtistID IS NULL;

AlbumStatus             AlbumCount
----------------------  ----------
With Artist             347       
                                  
Albums Without Artists  0         


In [36]:
%%script sqlite3 --column --header chinook.db
